In [1]:
import pandas

In [2]:
import os
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os 
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
'from livelossplot.keras import PlotLossesCallback'

'from livelossplot.keras import PlotLossesCallback'

In [3]:
import pandas as pd 
df = pd.read_csv('vinted_data.csv')
df

,Id,Titre,Source,Couleur
0,338907174,Chemise_The_Kooples_rose_ple,https://images.vinted.net/thumbs/310x430/05908...,#E6DACD
1,400207950,T-shirt_blanc_neuf_XL,https://images.vinted.net/thumbs/310x430/06821...,#F3ECE4
2,393538053,Blouse__Haut_Grace_and_Mila_noire_taille_S,https://images.vinted.net/thumbs/310x430/057d7...,#BBBDBE
3,400227021,Chemise_Aigle,https://images.vinted.net/thumbs/310x430/05187...,#EEECED
4,398870257,T-shirt_Replay,https://images.vinted.net/thumbs/310x430/0564a...,#E2E0E0
...,...,...,...,...
24599,400783629,Robe_noire_soire_1_Cache_Cache,https://images.vinted.net/thumbs/310x430/06e56...,#C2C2C2
24600,400783485,Mini_jupe_a_sequins_rose_et_beige_Pimkie,https://images.vinted.net/thumbs/310x430/06f40...,#D1CBCC
24601,400783446,Jupe_blanche_motif_zbr,https://images.vinted.net/thumbs/310x430/06fd0...,#ECECEC
24602,400783412,Superbe_jupe_cop_copine,https://images.vinted.net/thumbs/310x430/070c4...,#C2C5C7


In [4]:
batch_size = 32

In [5]:
from keras_preprocessing.image import ImageDataGenerator

TRAIN_DIR = 'data/vinted'
TEST_DIR = 'dating/vinted-val'


train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=45,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = .2)

test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split = .2)

train_generator = train_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(96,96),
    subset='training'
)

val_generator = test_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(96,96),
    subset='validation'
)

classes = len(train_generator.class_indices)
print(train_generator.class_indices)

Found 19681 images belonging to 11 classes.
Found 4915 images belonging to 11 classes.
{'Baskets': 0, 'Bottines': 1, 'Chemises': 2, 'Hauts & Tee-shirts': 3, 'Jupes': 4, 'Manteaux & vestes': 5, 'Pantalons': 6, 'Robes': 7, 'Sacs': 8, 'Sandales': 9, 'Sweats et sweats ├á capuche': 10}


In [ ]:
label_class = ['Sacs', 'Sandales', 'Baskets', 'Pantalons', 'Jupes', 'Robes', 'Chemises', 'Manteaux & vestes', 'Sweats et sweats à capuche', 'Bottines']

TRAIN_DIR = 'data/vinted'

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=45,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = .2)

test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split = .2)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(96,96),
    subset='training'
)

val_generator = test_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(96,96),
    subset='validation'
)

classes = len(train_generator.class_indices)
print(train_generator.class_indices)



# create the base pre-trained model
base_model = MobileNetV2(input_shape=(96, 96, 3),classes=11, include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(11, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
print(len(model.layers))
for layer in model.layers[:120]:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','mae', 'categorical_accuracy'])
model.summary()
                 
                 
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from math import ceil

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=ceil(0.8 * (df.size / batch_size)),

    validation_data=val_generator,
    validation_steps=ceil(0.2 * (df.size / batch_size)),
    epochs=40,
    shuffle=True,
    verbose=1
)
history

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
158
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 97, 97, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
_________

block_9_project_BN (BatchNormal (None, 6, 6, 64)     256         block_9_project[0][0]            
__________________________________________________________________________________________________
block_9_add (Add)               (None, 6, 6, 64)     0           block_8_add[0][0]                
                                                                 block_9_project_BN[0][0]         
__________________________________________________________________________________________________
block_10_expand (Conv2D)        (None, 6, 6, 384)    24576       block_9_add[0][0]                
__________________________________________________________________________________________________
block_10_expand_BN (BatchNormal (None, 6, 6, 384)    1536        block_10_expand[0][0]            
__________________________________________________________________________________________________
block_10_expand_relu (ReLU)     (None, 6, 6, 384)    0           block_10_expand_BN[0][0]         
__________

Epoch 1/40
2461/2461 [==============================] - 2246s 913ms/step - loss: 1.2053 - acc: 0.5913 - mean_absolute_error: 0.0948 - categorical_accuracy: 0.5913 - val_loss: 2.0060 - val_acc: 0.4439 - val_mean_absolute_error: 0.1124 - val_categorical_accuracy: 0.4439
Epoch 2/40
2461/2461 [==============================] - 2185s 888ms/step - loss: 1.0071 - acc: 0.6503 - mean_absolute_error: 0.0833 - categorical_accuracy: 0.6503 - val_loss: 1.9664 - val_acc: 0.4798 - val_mean_absolute_error: 0.1052 - val_categorical_accuracy: 0.4798
Epoch 3/40
2461/2461 [==============================] - 2171s 882ms/step - loss: 0.9265 - acc: 0.6763 - mean_absolute_error: 0.0777 - categorical_accuracy: 0.6763 - val_loss: 1.4641 - val_acc: 0.5882 - val_mean_absolute_error: 0.0832 - val_categorical_accuracy: 0.5882
Epoch 4/40
2461/2461 [==============================] - 2188s 889ms/step - loss: 0.8575 - acc: 0.6995 - mean_absolute_error: 0.0728 - categorical_accuracy: 0.6995 - val_loss: 1.4178 - val_acc: 

In [17]:
from tensorflow.keras.models import load_model

MODEL_PATH = 'model.h5'

model = load_model(MODEL_PATH)

In [20]:
import numpy as np
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
test_image = image.load_img('06776_2En87kuJw7R2s8tVo5w74yb4.jpeg', target_size = (96, 96))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)
#pred_class = decode_predictions(result, top=1) 


[[2.4379005e-03 2.8102801e-04 3.7469065e-01 1.3823749e-01 2.7043894e-01
  5.5831214e-03 8.8833362e-02 3.0315069e-02 5.8049761e-02 1.1113597e-02
  2.0019133e-02]]


In [22]:
words_to_search = ['Sacs', 'Sandales', 'Baskets', 'Pantalons', 'Jupes', 'Robes', 'Chemises', 'Manteaux & vestes', 'Sweats et sweats à capuche', 'Bottines']

# find the index of the class with maximum score
pred = result.argmax(axis=-1)

# print the label of the class with maximum score
print(words_to_search[pred[0]])

Chemises


In [ ]:
print('Baskets')